In [1]:
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor 
from sklearn.neighbors import KNeighborsRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
# Create your connection.
cnx = sqlite3.connect('database.sqlite')
df = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)
df.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [3]:
df.shape

(183978, 42)

### Since the variable we are trying to predict (overall rating) is a continuous variable, we have a regression problem.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   183978 non-null  int64  
 1   player_fifa_api_id   183978 non-null  int64  
 2   player_api_id        183978 non-null  int64  
 3   date                 183978 non-null  object 
 4   overall_rating       183142 non-null  float64
 5   potential            183142 non-null  float64
 6   preferred_foot       183142 non-null  object 
 7   attacking_work_rate  180748 non-null  object 
 8   defensive_work_rate  183142 non-null  object 
 9   crossing             183142 non-null  float64
 10  finishing            183142 non-null  float64
 11  heading_accuracy     183142 non-null  float64
 12  short_passing        183142 non-null  float64
 13  volleys              181265 non-null  float64
 14  dribbling            183142 non-null  float64
 15  curve            

In [5]:
df.describe()

,id,player_fifa_api_id,player_api_id,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
count,183978.00000,183978.000000,183978.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000,181265.000000,...,181265.000000,183142.000000,183142.000000,183142.000000,181265.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000
mean,91989.50000,165671.524291,135900.617324,68.600015,73.460353,55.086883,49.921078,57.266023,62.429672,49.468436,...,57.873550,55.003986,46.772242,50.351257,48.001462,14.704393,16.063612,20.998362,16.132154,16.441439
std,53110.01825,53851.094769,136927.840510,7.041139,6.592271,17.242135,19.038705,16.488905,14.194068,18.256618,...,15.144086,15.546519,21.227667,21.483706,21.598778,16.865467,15.867382,21.452980,16.099175,17.198155
min,1.00000,2.000000,2625.000000,33.000000,39.000000,1.000000,1.000000,1.000000,3.000000,1.000000,...,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,45995.25000,155798.000000,34763.000000,64.000000,69.000000,45.000000,34.000000,49.000000,57.000000,35.000000,...,49.000000,45.000000,25.000000,29.000000,25.000000,7.000000,8.000000,8.000000,8.000000,8.000000
50%,91989.50000,183488.000000,77741.000000,69.000000,74.000000,59.000000,53.000000,60.000000,65.000000,52.000000,...,60.000000,57.000000,50.000000,56.000000,53.000000,10.000000,11.000000,12.000000,11.000000,11.000000
75%,137983.75000,199848.000000,191080.000000,73.000000,78.000000,68.000000,65.000000,68.000000,72.000000,64.000000,...,69.000000,67.000000,66.000000,69.000000,67.000000,13.000000,15.000000,15.000000,15.000000,15.000000
max,183978.00000,234141.000000,750584.000000,94.000000,97.000000,95.000000,97.000000,98.000000,97.000000,93.000000,...,97.000000,96.000000,96.000000,95.000000,95.000000,94.000000,93.000000,97.000000,96.000000,96.000000


### Dropping columns that we don't need for our model / Creating new column based on other columns

In [6]:
df.drop(["id", "date", "player_fifa_api_id", "player_api_id"], axis=1, inplace=True)

### Looking at unique values of each column / Handling strange values

In [7]:
for col in df.columns:
    print(col + " : " + str(df[col].unique()))

overall_rating : [67. 62. 61. 74. 73. 71. 70. 69. 68. 65. 64. 54. 51. 52. 47. 53. 66. 59.
 75. 72. 76. 78. 77. 79. 60. 80. 81. 82. 84. 48. 63. 83. 55. 58. 50. 56.
 nan 49. 57. 42. 46. 45. 85. 44. 86. 89. 87. 88. 91. 40. 90. 41. 43. 38.
 93. 92. 39. 33. 36. 37. 35. 94.]
potential : [71. 66. 65. 76. 75. 77. 78. 79. 80. 68. 64. 60. 67. 70. 72. 69. 82. 73.
 74. 81. 83. 86. 84. 85. 87. 90. 56. 57. 63. 62. 61. nan 59. 55. 58. 53.
 89. 54. 88. 52. 91. 92. 93. 51. 46. 44. 50. 47. 45. 95. 94. 48. 49. 42.
 97. 96. 39.]
preferred_foot : ['right' 'left' None]
attacking_work_rate : ['medium' 'high' None 'low' 'None' 'le' 'norm' 'stoc' 'y']
defensive_work_rate : ['medium' 'high' 'low' '_0' None '5' 'ean' 'o' '1' 'ormal' '7' '2' '8' '4'
 'tocky' '0' '3' '6' '9' 'es']
crossing : [49. 48. 80. 79. 78. 77. 74. 58. 57. 22. 64. 65. 67. 69. 72. 73. 63. 56.
 59. 60. 46. 45. 75. 76. 47. 82. 71. 12. 62. 26. 70. 66. 40. 53. 55. 54.
 61. 39. 41. 42. 52. 68. nan 35. 34. 50. 38. 84. 81. 25. 11.  6. 23. 29.
 36. 44

### Check if types of columns are correct

In [8]:
df.dtypes

overall_rating         float64
potential              float64
preferred_foot          object
attacking_work_rate     object
defensive_work_rate     object
crossing               float64
finishing              float64
heading_accuracy       float64
short_passing          float64
volleys                float64
dribbling              float64
curve                  float64
free_kick_accuracy     float64
long_passing           float64
ball_control           float64
acceleration           float64
sprint_speed           float64
agility                float64
reactions              float64
balance                float64
shot_power             float64
jumping                float64
stamina                float64
strength               float64
long_shots             float64
aggression             float64
interceptions          float64
positioning            float64
vision                 float64
penalties              float64
marking                float64
standing_tackle        float64
sliding_

In [9]:
# Object datatypes must have only categorical/string values (object datatype means that it can have different datatype values)
for i in range(len(df.dtypes)):
    if df.dtypes[i] == "object":
        print(df.dtypes.index[i])

preferred_foot
attacking_work_rate
defensive_work_rate


In [10]:
for i in range(len(df.dtypes)):
    if df.dtypes[i] == "object":
        print(df.dtypes.index[i] + " : ")
        print(df[df.dtypes.index[i]].value_counts())
        print()

preferred_foot : 
right    138409
left      44733
Name: preferred_foot, dtype: int64

attacking_work_rate : 
medium    125070
high       42823
low         8569
None        3639
norm         348
y            106
le           104
stoc          89
Name: attacking_work_rate, dtype: int64

defensive_work_rate : 
medium    130846
high       27041
low        18432
_0          2394
o           1550
1            441
ormal        348
2            342
3            258
5            234
7            217
0            197
6            197
9            152
4            116
es           106
ean          104
tocky         89
8             78
Name: defensive_work_rate, dtype: int64



In [11]:
df["preferred_foot"].unique()

array(['right', 'left', None], dtype=object)

In [12]:
df["attacking_work_rate"].unique()
# We have "None" value which is different from None so we must handle it.

array(['medium', 'high', None, 'low', 'None', 'le', 'norm', 'stoc', 'y'],
      dtype=object)

In [13]:
df["attacking_work_rate"].replace("None", df["attacking_work_rate"].mode()[0], inplace=True)
df["attacking_work_rate"].unique()

array(['medium', 'high', None, 'low', 'le', 'norm', 'stoc', 'y'],
      dtype=object)

In [14]:
df["defensive_work_rate"].unique()

array(['medium', 'high', 'low', '_0', None, '5', 'ean', 'o', '1', 'ormal',
       '7', '2', '8', '4', 'tocky', '0', '3', '6', '9', 'es'],
      dtype=object)

### Handling Missing Values

In [15]:
df.isna().sum().values.any()

True

In [16]:
# Check for the number of NAN and None values in each column.
df.isna().sum()

overall_rating          836
potential               836
preferred_foot          836
attacking_work_rate    3230
defensive_work_rate     836
crossing                836
finishing               836
heading_accuracy        836
short_passing           836
volleys                2713
dribbling               836
curve                  2713
free_kick_accuracy      836
long_passing            836
ball_control            836
acceleration            836
sprint_speed            836
agility                2713
reactions               836
balance                2713
shot_power              836
jumping                2713
stamina                 836
strength                836
long_shots              836
aggression              836
interceptions           836
positioning             836
vision                 2713
penalties               836
marking                 836
standing_tackle         836
sliding_tackle         2713
gk_diving               836
gk_handling             836
gk_kicking          

In [17]:
for i in range(len(df.dtypes)):
    col = df.dtypes.index[i]
    if df.dtypes[i] == "object":
        df[col].fillna(df[col].mode()[0], inplace=True)
    else:
        df[col].fillna(df[col].mean(), inplace=True)

In [18]:
# NAN and None will be set to 0
df.isna().sum()

overall_rating         0
potential              0
preferred_foot         0
attacking_work_rate    0
defensive_work_rate    0
crossing               0
finishing              0
heading_accuracy       0
short_passing          0
volleys                0
dribbling              0
curve                  0
free_kick_accuracy     0
long_passing           0
ball_control           0
acceleration           0
sprint_speed           0
agility                0
reactions              0
balance                0
shot_power             0
jumping                0
stamina                0
strength               0
long_shots             0
aggression             0
interceptions          0
positioning            0
vision                 0
penalties              0
marking                0
standing_tackle        0
sliding_tackle         0
gk_diving              0
gk_handling            0
gk_kicking             0
gk_positioning         0
gk_reflexes            0
dtype: int64

### Separate the data into X (independent variables) and y (dependent variable)

In [19]:
X = df.drop(["overall_rating"], axis=1).copy()
y = df["overall_rating"].copy()

In [20]:
X = pd.get_dummies(X, columns=["preferred_foot", "attacking_work_rate", "defensive_work_rate"])

In [21]:
y.unique()

array([67.        , 62.        , 61.        , 74.        , 73.        ,
       71.        , 70.        , 69.        , 68.        , 65.        ,
       64.        , 54.        , 51.        , 52.        , 47.        ,
       53.        , 66.        , 59.        , 75.        , 72.        ,
       76.        , 78.        , 77.        , 79.        , 60.        ,
       80.        , 81.        , 82.        , 84.        , 48.        ,
       63.        , 83.        , 55.        , 58.        , 50.        ,
       56.        , 68.60001529, 49.        , 57.        , 42.        ,
       46.        , 45.        , 85.        , 44.        , 86.        ,
       89.        , 87.        , 88.        , 91.        , 40.        ,
       90.        , 41.        , 43.        , 38.        , 93.        ,
       92.        , 39.        , 33.        , 36.        , 37.        ,
       35.        , 94.        ])

In [22]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [23]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)

x_test = scaler.transform(x_test)

In [24]:
# GridSearchCV on Linear Regression

param = {  'fit_intercept': [True, False], 
            'normalize': [True, False], 
            'copy_X': [True, False]
            }

gridsearch = GridSearchCV(estimator=LinearRegression(), param_grid=param, 
                          cv=5, n_jobs=-1, verbose=3)
gridsearch.fit(x_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   12.0s finished


GridSearchCV(cv=5, estimator=LinearRegression(), n_jobs=-1,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             verbose=3)

In [25]:
gridsearch.best_params_

{'copy_X': True, 'fit_intercept': True, 'normalize': False}

In [26]:
lr = LinearRegression(fit_intercept=gridsearch.best_params_["fit_intercept"], normalize=gridsearch.best_params_["normalize"],
                      copy_X=gridsearch.best_params_["copy_X"])
lr.fit(x_train, y_train)
pred_test = lr.predict(x_test)
print(r2_score(y_test, pred_test))

0.8425596617951191


In [28]:
# GridSearchCV on DecisionTreeRegressor

grid_param = { "max_depth" : range(1,21),
               "min_samples_leaf" : [1, 5, 10, 20, 50, 100]
              }

gridsearch = GridSearchCV(DecisionTreeRegressor(), param_grid=grid_param, cv=5, n_jobs=-1, verbose=3)
gridsearch.fit(x_train, y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   27.5s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 504 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:  5.7min finished


GridSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'max_depth': range(1, 21),
                         'min_samples_leaf': [1, 5, 10, 20, 50, 100]},
             verbose=3)

In [29]:
gridsearch.best_params_

{'max_depth': 20, 'min_samples_leaf': 1}

In [30]:
dt = DecisionTreeRegressor(max_depth=gridsearch.best_params_["max_depth"], 
                           min_samples_leaf=gridsearch.best_params_["min_samples_leaf"])
dt.fit(x_train, y_train)
pred_test = dt.predict(x_test)
print(r2_score(y_test, pred_test))

0.9572077542544752


Decision Tree Regressor has higher accuracy so we choose this model

Since we are testing using negative mean squared error we choose the model with the highest negative mean squared error. In our case the best model is Decision Tree. Let's do some hyperparameter tuning using GridSearchCV to find a better decision tree.